In [ ]:
import yaml
from pathlib import Path
import sys
import numpy as np
import matplotlib.pyplot as plt

# Get project root and add to path
notebook_dir = Path.cwd()
project_root = Path('pytorch/crypto/classifier')
sys.path.insert(0, str(project_root))

# Import project modules
from src.data_loader import download_crypto_data, create_price_sequences
from src.gpu_renderer import GPURenderer
from src.utils import check_gpu_availability

# Load config
with open(project_root / 'config' / 'config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Check GPU availability FIRST
gpu_available, backend = check_gpu_availability()
print(f"Compute Backend: {backend}")

if not gpu_available:
    print("❌ GPU (CuPy) is required but not available")
    print("This notebook requires GPU rendering. Please ensure CuPy is installed and CUDA is available.")
    print("Install CuPy: pip install cupy-cuda11x (or appropriate CUDA version)")
    raise RuntimeError("GPU not available - cannot proceed with GPU rendering")

# Download/load data
data_config = config['data']
cache_dir = project_root / config['paths']['raw_data']

data = download_crypto_data(
    symbol=data_config['symbol'],
    interval=data_config['interval'],
    start_date_str=data_config['start_date'],
    end_date_str=data_config['end_date'],
    cache_dir=str(cache_dir)
)

print(f"Loaded {len(data)} data points")

# Create sequences
seq_len = config['image']['seq_len']
closing_prices = data.values
sequences = create_price_sequences(closing_prices, seq_len)

print(f"Created {len(sequences)} sequences")

# Render first image using the EXACT SAME batch GPU method as pipeline
renderer = GPURenderer()
resolution = config['image']['resolution']
line_width = config['image']['line_width']

# Use batch rendering (same as pipeline) - pass single sequence as batch of 1
batch_images = renderer.render_batch_gpu(sequences[0:1], resolution, line_width)
img = batch_images[0]  # Extract the single image

print(f"Image shape (H x W): {img.shape}")
print(f"Pixel value range: [{img.min():.3f}, {img.max():.3f}]")
print(f"Data type: {img.dtype}")

# Display the EXACT image that gets fed to the model (GPU-rendered, matplotlib for display)
print("GPU-rendered image statistics:")
print(f"Image shape (H x W): {img.shape}")
print(f"Pixel value range: [{img.min():.3f}, {img.max():.3f}]")
print(f"Data type: {img.dtype}")
print(f"Unique pixel values: {len(np.unique(img))}")
print(f"Black pixels (0.0): {np.sum(img == 0.0)}")
print(f"White pixels (1.0): {np.sum(img == 1.0)}")
print(f"Gray pixels (0.0 < x < 1.0): {np.sum((img > 0.0) & (img < 1.0))}")

# Convert GPU image to CPU for matplotlib display
import cupy as cp
img_cpu = cp.asnumpy(img) if hasattr(img, 'get') else img

# Display with matplotlib (for visualization only)
plt.figure(figsize=(12, 7))
plt.imshow(img_cpu, cmap='gray', aspect='auto')
plt.axis('off')  # Remove all axes, labels, and ticks
plt.tight_layout(pad=0)  # Remove padding
plt.show()

# Save as raw image file for external viewing (optional)
np.save('gpu_rendered_image.npy', img_cpu)
print(f"\nSaved raw image as 'gpu_rendered_image.npy'")